In [1]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator as Generator 
import model_definitions

### Open input file

In [2]:
class PcPath:
    def __init__(self,physic,commPhy):
        self.physic=physic
        self.commPhy=commPhy
        


DANIELE=PcPath(r"C:\Users\dansp\OneDrive\Desktop\Tesi\Logs\OneCar_db.txt",
               r"C:\Users\dansp\OneDrive\Desktop\Tesi\Logs\PhysycAndCommand_db.txt")
POLI=PcPath(r"D:\Users\Students\Daniele\records\OneCar_db.txt",
           r"D:\Users\Students\Daniele\records\PhysycAndCommand_db.txt")

PCS=[DANIELE,POLI]
PC_INDEX=1

COLUMNS_PHY=["Player", "X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC_X","ACC_Z","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME"]

COLUMNS_COMM_PHY=["Player", "X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC_X","ACC_Z","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME","MOVE_X","MOVE_Z","BREAKING"]


In [3]:
SEQUENCE_LENGTH=50
ONE_CAR=True
COMM_PHY=False
#PATH=r"C:\Users\dansp\OneDrive\Desktop\Tesi\SCRITTURA\OneCar_tack1_db.txt"
BATCH_SIZE=256*2
FEATURES=7
FEATURES_PREDICTED=6
DISCARD=9

if(COMM_PHY):
    PATH=PCS[PC_INDEX].commPhy
    FEATURES +=3 #adding the command to the features 
    COLUMNS=COLUMNS_COMM_PHY
else:
    PATH=PATH=PCS[PC_INDEX].physic
    COLUMNS=COLUMNS_PHY

In [4]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
Generator.DISCARD=DISCARD
Generator.COLUMNS=COLUMNS
Generator.COMM_PHY=COMM_PHY
train,val,test=Generator.single_care_dataframe(PATH)

In [5]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

def SubtractAndBuildDF(df):
    dfs=[]
    for i in range(len(df)):
        dfs.append(SubtractDF(df[i]))
        #dfs.append(df[i])
    united=dfs[0]
    for i in range(1,len(df)):
        united=pd.concat([united,dfs[i] ],ignore_index=True)
    return united

df_train= SubtractAndBuildDF(train)
df_val= SubtractAndBuildDF(val)
df_test= SubtractAndBuildDF(test)

In [6]:
df_train.head()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
0,0.00000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,4,32,-0.124001,0.389333,0.2,0,0
1,-0.00066,0.39131,-0.030470,3.676403,-0.0278,-0.024275,-1.523513,16.220290,4,32,-0.124009,0.394551,0.2,0,0
2,-0.20325,0.98584,-0.372497,2.119079,-5.7426,-0.118653,35.271613,-13.825137,4,32,-0.126719,0.407695,0.2,0,0
3,-0.27722,1.33862,-0.699331,1.527281,-3.7373,0.280035,-21.201270,1.749897,4,32,-0.130415,0.425544,0.2,0,0
4,-0.19885,1.60254,-0.089037,1.494527,1.7791,-0.131975,-14.416056,6.210950,4,32,-0.133066,0.446911,0.2,0,0


In [7]:
def check_nan_values(dataframe):
    nan_values = dataframe.isnull().sum()
    if nan_values.sum() == 0:
        print("No NaN values found in the DataFrame.")
    else:
        print("NaN values found in the DataFrame:")
        for col, count in nan_values.items():
            if count > 0:
                print(f"Column '{col}' has {count} NaN value(s) at indexes:")
                nan_indexes = dataframe[dataframe[col].isnull()].index.tolist()
                print(nan_indexes)
                return nan_indexes
                
ind=check_nan_values(df_test)

No NaN values found in the DataFrame.


In [8]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,100000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,100000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,100000)


Length: 2720316 races: 84 n batches: 2678316 / 512
Length: 1286892 races: 42 n batches: 1265892 / 512
Length: 1303610 races: 42 n batches: 1282610 / 512


### Model definition

#### Train and print

In [9]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [41]:
NAME=f"MLP_DISCARD_{DISCARD}_DELTA"
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,
                                                  gen_val,
                                                  learning_rate=0.0001,
                                                  epoch=1,
                                                  verbose=1,
                                                  name=NAME
                                                  ,path=f"new_split_tile_info_CommPhy_DISCARD{DISCARD}")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
#tf.saved_model.save(mlp_ph,f'saved_model/{NAME}')

2267/2268 [============================>.] - ETA: 0s - loss: 14.1542 - mean_absolute_error: 2.1590 - accuracy: 1.4359e-07INFO:tensorflow:Assets written to: ./new_split_tile_info_CommPhy_DISCARD9/MLP_DISCARD_9_DELTA/tmp\checkpoint\assets


INFO:tensorflow:Assets written to: ./new_split_tile_info_CommPhy_DISCARD9/MLP_DISCARD_9_DELTA/tmp\checkpoint\assets


2268/2268 [==============================] - 255s 112ms/step - loss: 14.1497 - mean_absolute_error: 2.1586 - accuracy: 1.4353e-07 - val_loss: 6.3993 - val_mean_absolute_error: 1.3000 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Loss: 6.83797550201416 MAE: 1.299050211906433 Accuracy: 2.8730877943417e-07


In [ ]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

In [ ]:
lr =1e-4
units=512
cells=6
normalization=False
mlp_units=64
mlp_cells=3
dropout=0.4
name=f"LSTM{cells}_{units}_{lr}_{DISCARD}_MLP_{mlp_cells}_{mlp_units}_DROPOUT_{dropout}"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(
    gen_train,
    gen_val,
    learning_rate=lr,
    patience=15,
    name=name,
    path=f"new_split_tile_info_DISCARD{DISCARD}",
    epoch=100, 
    dropout=dropout,
    cells=cells,
    units=units,
    mlp_units=mlp_units,
    mlp_cells=mlp_cells,
    normalization=normalization,
    verbose=1
)
ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
#tf.saved_model.save(lstm_ph,f'saved_model/{name}')

Epoch 1/100
5229/5229 [==============================] - ETA: 0s - loss: 1.9480 - mean_absolute_error: 0.6611 - accuracy: 1.8676e-07

INFO:tensorflow:Assets written to: ./new_split_tile_info_DISCARD9/LSTM6_512_0.0001_9_MLP_3_64_DROPOUT_0.4/tmp\checkpoint\assets


INFO:tensorflow:Assets written to: ./new_split_tile_info_DISCARD9/LSTM6_512_0.0001_9_MLP_3_64_DROPOUT_0.4/tmp\checkpoint\assets


5229/5229 [==============================] - 1072s 204ms/step - loss: 1.9480 - mean_absolute_error: 0.6611 - accuracy: 1.8676e-07 - val_loss: 8.2946 - val_mean_absolute_error: 1.5004 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/100
5229/5229 [==============================] - ETA: 0s - loss: 1.3430 - mean_absolute_error: 0.5351 - accuracy: 5.6028e-07

INFO:tensorflow:Assets written to: ./new_split_tile_info_DISCARD9/LSTM6_512_0.0001_9_MLP_3_64_DROPOUT_0.4/tmp\checkpoint\assets


...

INFO:tensorflow:Assets written to: ./new_split_tile_info_DISCARD9/LSTM6_512_0.0001_9_MLP_3_64_DROPOUT_0.4/tmp\checkpoint\assets


5229/5229 [==============================] - 1023s 195ms/step - loss: 0.7026 - mean_absolute_error: 0.3713 - accuracy: 1.6808e-06 - val_loss: 2.6087 - val_mean_absolute_error: 0.7670 - val_accuracy: 1.1861e-06 - lr: 1.0000e-04
Epoch 15/100
5229/5229 [==============================] - ETA: 0s - loss: 0.6877 - mean_absolute_error: 0.3671 - accuracy: 7.4704e-07

INFO:tensorflow:Assets written to: ./new_split_tile_info_DISCARD9/LSTM6_512_0.0001_9_MLP_3_64_DROPOUT_0.4/tmp\checkpoint\assets


INFO:tensorflow:Assets written to: ./new_split_tile_info_DISCARD9/LSTM6_512_0.0001_9_MLP_3_64_DROPOUT_0.4/tmp\checkpoint\assets


5229/5229 [==============================] - 1023s 195ms/step - loss: 0.6877 - mean_absolute_error: 0.3671 - accuracy: 7.4704e-07 - val_loss: 2.3236 - val_mean_absolute_error: 0.7139 - val_accuracy: 6.5895e-07 - lr: 1.0000e-04
Epoch 16/100
5229/5229 [==============================] - 1008s 193ms/step - loss: 0.6733 - mean_absolute_error: 0.3633 - accuracy: 1.1828e-06 - val_loss: 2.9369 - val_mean_absolute_error: 0.8041 - val_accuracy: 9.2253e-07 - lr: 1.0000e-04
Epoch 17/100
5229/5229 [==============================] - 1006s 192ms/step - loss: 0.6612 - mean_absolute_error: 0.3596 - accuracy: 1.0583e-06 - val_loss: 2.7008 - val_mean_absolute_error: 0.7672 - val_accuracy: 9.2253e-07 - lr: 1.0000e-04
Epoch 18/100
5198/5229 [============================>.] - ETA: 4s - loss: 0.6518 - mean_absolute_error: 0.3564 - accuracy: 1.0020e-06

In [43]:
lstm_ph.summary()

Model: "LSTM_NN1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20, 10)]          0         
                                                                 
 lstm_4 (LSTM)               (None, 20, 64)            19200     
                                                                 
 batch_normalization_2 (Batc  (None, 20, 64)           256       
 hNormalization)                                                 
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 flatten_2 (Flatten)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                          

In [ ]:
test[1].shape

In [ ]:
y_pred = lstm_ph.predict(gen_test)


In [ ]:
y_pred

In [ ]:
ev

In [ ]:
lr =1e-3
units=512
cells=8
normalization=False
name=f"LSTM{cells}_{units}_{lr}_DISCARD_{DISCARD}_DELTA"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=50,name=name,path="logs",
                                                     epoch=100, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

In [ ]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

In [ ]:
model=model_definitions.MLP_Model(32)
model.load_weights("saved_model\MLP0001_PosAssoluta")

In [ ]:
model.summary()

In [ ]:
zero=gen_test[0]
toP=zero[0][0]
pred=zero[1][0]
print(toP)
print(pred)
model.predict(toP)

In [ ]:
import tf2onnx
import onnx
input_signature=(tf.TensorSpec([None,SEQUENCE_LENGTH,FEATURES],tf.float32,name='x'))
onnx_model,_=tf2onnx.converter.from_keras(model,input_signature,opset=9)

In [ ]:
races=united["RACE"].max()
print(races*0.5)
training=united.loc[united['RACE'] <= races*0.5]
training.tail(10)

In [ ]:
t=df_train["ROT"].idxmax()
df_train.iloc[t-2:t+3]